# Cleaning + feature engineering

## Imports

In [131]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [2]:
os.getcwd();

In [3]:
info_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/info_10k.csv'  #redacted
credits_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/credits_10k.csv'  #redacted
releases_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/releases_10k.csv'  #redacted

In [4]:
#info table contains basic film details like genre, budget, revenue, language, runtime, etc.
info_df = pd.read_csv(info_file)

#credits table contains cast and crew
credits_df = pd.read_csv(credits_file)

#releases table contains info on which countries each film was released in
releases_df = pd.read_csv(releases_file)

## Cleaning + new features

#### Because of the way data was collected, last row of each table has the headers. Fixing that:

In [5]:
info_df.columns = info_df.iloc[-1:, :].squeeze()
info_df = info_df.iloc[:-1, :]

In [6]:
credits_df.columns = credits_df.iloc[-1:, :].squeeze()
credits_df = credits_df.iloc[:-1, :]

In [7]:
releases_df.columns = releases_df.iloc[-1:, :].squeeze()
releases_df = releases_df.iloc[:-1, :]

### Cleaning info table

#### Getting each film's genre:

In [8]:
def get_first_genre(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first genre from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            return lst_of_dicts[i]['name']
    else:
        pass

In [9]:
info_df['genre0'] = info_df['genres'].apply(get_first_genre)  #func in hidden cell above

#### Dropping unnecessary rows and columns from info table:

In [10]:
#making sure relevant numerical columns are int/float type
info_df = info_df.astype({"budget": float, "id": int, "revenue": float, "runtime": float})

In [11]:
#filtering for only non-adult films,
# with revenue & budget > 0,
# original language english,
# runtime >= 80 minutes,
# and genre is not documentary.

info_df = info_df[(info_df['adult'] == 'False') & \
        (info_df['budget'] > 0) & \
        (info_df['revenue'] > 0) & \
        (info_df['original_language'] == 'en') & \
        (info_df['runtime'] >= 80.0) & \
        (info_df['genre0'] != 'Documentary')]

In [12]:
#dropping columns from info_df that won't be needed for EDA
drop_cols = ['adult', 'backdrop_path', 'belongs_to_collection', 
             'genres', 'homepage', 'imdb_id', 'original_language', 'original_title', 
             'overview', 'popularity', 'poster_path', 'production_companies', 
             'production_countries', 'spoken_languages', 'status', 'tagline', 'video', 
             'vote_average', 'vote_count']

info_df = info_df.drop(drop_cols, axis=1)

### a few more cols to be dropped later for modelling, after EDA, feat. engineering, & joins: 
### ['id', 'release_date', 'runtime' (possibly), 'title', 'genre' (possibly)]

#### Creating columns for (a) amount of profit or loss (revenue minus budget) and (b) whether each movie profited (binary, whether revenue > budget):

In [13]:
info_df['profit'] = info_df['revenue'] - info_df['budget']

In [14]:
info_df['made_money'] = info_df['profit'] > 0
info_df['made_money'] = info_df['made_money'].astype('int')

In [56]:
# print(info_df.info())
info_df.head()

4962,budget,id,release_date,revenue,runtime,title,genre0,profit,made_money
1,4000000.0,5,1995-12-09,4300000.0,98.0,Four Rooms,Crime,300000.0,1
2,21.0,6,1993-10-15,12.0,110.0,Judgment Night,Action,-9.0,0
5,11000000.0,11,1977-05-25,775398007.0,121.0,Star Wars,Adventure,764398007.0,1
6,94000000.0,12,2003-05-30,940335536.0,100.0,Finding Nemo,Animation,846335536.0,1
7,55000000.0,13,1994-07-06,677945399.0,142.0,Forrest Gump,Comedy,622945399.0,1


### Cleaning credits table

#### Appending columns with top billed actors to the credits table:

In [24]:
def get_cast_member(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    actor (str): name of the actor corresponding to the billing 
    number in this function (lower billing number = more prominent
    role; 0 = star of the movie, 1 = costar, etc.)
    '''
    lst_of_dicts = eval(lod_str)
    billing = 4  #increment this line manually and rerun for desired # of actors, starting at 0
    if len(lst_of_dicts) > billing:
        dict_of_interest = lst_of_dicts[billing]
        actor = dict_of_interest['name']
        return actor
    else:
        pass

In [25]:
# DO NOT FORGET to manually increment the new col name here, e.g. 'cast0', 'cast1', etc.
credits_df['cast4'] = credits_df['cast'].apply(get_cast_member)

#### Appending columns with director and screenwriters to the credits table:

In [26]:
def get_director(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first director from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Director':
                return lst_of_dicts[i]['name']
    else:
        pass

In [27]:
credits_df['director0'] = credits_df['crew'].apply(get_director)  #func in hidden cell above

In [28]:
def get_first_writer(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first screenwriter from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Screenplay' or \
            lst_of_dicts[i]['job'] == 'Writer' or \
            lst_of_dicts[i]['job'] == 'Author':
                return lst_of_dicts[i]['name']
    else:
        pass

In [29]:
credits_df['writer0'] = credits_df['crew'].apply(get_first_writer)  #func in hidden cell above

In [41]:
creds_drop_cols = ['cast', 'crew']
credits_df = credits_df.drop(creds_drop_cols, axis=1)

In [99]:
#making sure id column is int so dataframes can be merged later
credits_df = credits_df.astype({"id": int})

In [ ]:
# def get_second_writer(lod_str):
#     '''
#     Function for use with pandas apply method.
#     --------
#     INPUT
#     lod_str (str): a list of dictionaries in the form of a string
#     --------
#     OUTPUT
#     (str): name of the second screenwriter from the list of dicts
#     '''
#     lst_of_dicts = eval(lod_str)
#     if len(lst_of_dicts) > 0:
#         count = 0
#         for i in range(len(lst_of_dicts)):
#             if lst_of_dicts[i]['job'] == 'Screenplay' and count == 0:
#                 count += 1
#             elif lst_of_dicts[i]['job'] == 'Screenplay' and count == 1:
#                 return lst_of_dicts[i]['name']
#     else:
#         pass

In [ ]:
# credits_df['writer1'] = credits_df['crew'].apply(get_second_writer)
### decided not to get second writer becuse relatively few films have one in the data

#### Dropping rows without at least 5 actors, or missing director, or missing writer:

In [47]:
credits_df = credits_df.dropna(subset=['cast4', 'director0', 'writer0'])

### Cleaning releases table

#### Getting number of countries each film was released in and dropping nulls:

In [75]:
def get_releases_count(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (int): number of countries/territories the film was released in
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        return len(lst_of_dicts)
    else:
        pass

In [76]:
releases_df['releases'] = releases_df['countries'].apply(get_releases_count)  #func in hidden cell above

In [80]:
releases_df = releases_df.dropna()

In [86]:
releases_df = releases_df.drop('countries', axis=1)

In [100]:
#making sure id column is int so dataframes can be merged later
releases_df = releases_df.astype({"id": int})

### Joining the tables

#### Note: when the keys to join on are pd columns, not indices, better to use merge method

In [101]:
step1 = info_df.merge(credits_df, on='id', how='inner')

In [148]:
df = step1.merge(releases_df, on='id', how='inner')

### Checking for outliers/faulty data (HAVE TO REEXAMINE THIS ON FULL DATA)

In [129]:
pd.set_option('float_format', '{:f}'.format)  #so that floats will print w/o scientific notat.

In [149]:
df.describe()  #checking for outliers/faulty data, looks like we have an issue in budget/revenue

,budget,id,revenue,runtime,profit,made_money,releases
count,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000
mean,38622310.793661,4280.022639,128294032.535576,114.152005,89671721.741915,0.801423,17.402975
std,40247799.140231,3758.514757,173168777.950504,21.962564,150251482.734078,0.399058,17.390917
min,1.000000,5.000000,5.000000,80.000000,-98301101.000000,0.000000,1.000000
25%,10000000.000000,790.250000,22000000.000000,99.000000,3019205.000000,1.000000,6.000000
50%,26000000.000000,2469.500000,65730000.000000,110.000000,36760682.000000,1.000000,10.000000
75%,55000000.000000,8871.750000,163718070.250000,124.000000,115568858.500000,1.000000,19.000000
max,380000000.000000,9992.000000,1845034188.000000,248.000000,1645034188.000000,1.000000,118.000000


In [150]:
df = df[df['budget'] > 1000] #dropping rows with faulty budget numbers

In [166]:
df = df[df['revenue'] > 10300] #dropping rows with faulty revenue numbers

In [167]:
df.describe()  #much better (the 12k revenue and 12k budget are not mistakes)

,budget,id,revenue,runtime,profit,made_money,releases
count,1535.000000,1535.000000,1535.000000,1535.000000,1535.000000,1535.000000,1535.000000
mean,38841753.859935,4281.883388,129138410.224104,114.202606,90296656.364169,0.803909,17.463844
std,40272976.340912,3758.806706,173477195.271823,21.999348,150567591.003754,0.397168,17.426354
min,12000.000000,5.000000,12000.000000,80.000000,-98301101.000000,0.000000,1.000000
25%,10000000.000000,791.000000,22430661.500000,99.000000,3684759.500000,1.000000,6.000000
50%,26000000.000000,2486.000000,66000000.000000,110.000000,37410000.000000,1.000000,10.000000
75%,55000000.000000,8873.000000,163938407.500000,124.000000,118302504.500000,1.000000,19.000000
max,380000000.000000,9992.000000,1845034188.000000,248.000000,1645034188.000000,1.000000,118.000000


In [193]:
# resetting index for easier for-looping later
df = df.reset_index().drop('index', axis=1)

In [195]:
print(len(df))
df.tail()

1535


,budget,id,release_date,revenue,runtime,title,genre0,profit,made_money,cast0,cast1,cast2,cast3,cast4,director,writer0,releases
1530,150000000.000000,9982,2005-11-04,314432665.000000,81.000000,Chicken Little,Animation,164432665.000000,1,Zach Braff,Joan Cusack,Steve Zahn,Garry Marshall,Amy Sedaris,Mark Dindal,Steve Bencich,21.000000
1531,85000000.000000,9986,2006-12-07,144000000.000000,97.000000,Charlotte's Web,Comedy,59000000.000000,1,Dakota Fanning,Julia Roberts,Steve Buscemi,John Cleese,Oprah Winfrey,Gary Winick,Karey Kirkpatrick,6.000000
1532,17500000.000000,9988,2006-08-25,19179969.000000,110.000000,Beerfest,Comedy,1679969.000000,1,Erik Stolhanske,Jay Chandrasekhar,Steve Lemme,Paul Soter,M.C. Gainey,Jay Chandrasekhar,Jay Chandrasekhar,6.000000
1533,30000000.000000,9989,2001-01-12,18195610.000000,108.000000,Antitrust,Action,-11804390.000000,0,Ryan Phillippe,Rachael Leigh Cook,Tim Robbins,Claire Forlani,Richard Roundtree,Peter Howitt,Howard Franklin,4.000000
1534,86000000.000000,9992,2006-12-13,107944236.000000,94.000000,Arthur and the Invisibles,Adventure,21944236.000000,1,Freddie Highmore,Mia Farrow,Ron Crawford,Penny Balfour,Jason Bateman,Luc Besson,Luc Besson,12.000000


In [224]:
df.head()

,budget,id,release_date,revenue,runtime,title,genre0,profit,made_money,cast0,cast1,cast2,cast3,cast4,director,writer0,releases
0,4000000.000000,5,1995-12-09,4300000.000000,98.000000,Four Rooms,Crime,300000.000000,1,Tim Roth,Antonio Banderas,Jennifer Beals,Madonna,Marisa Tomei,Allison Anders,Allison Anders,30.000000
1,11000000.000000,11,1977-05-25,775398007.000000,121.000000,Star Wars,Adventure,764398007.000000,1,Mark Hamill,Harrison Ford,Carrie Fisher,Peter Mayhew,Anthony Daniels,George Lucas,George Lucas,67.000000
2,94000000.000000,12,2003-05-30,940335536.000000,100.000000,Finding Nemo,Animation,846335536.000000,1,Albert Brooks,Ellen DeGeneres,Alexander Gould,Willem Dafoe,Brad Garrett,Andrew Stanton,Andrew Stanton,62.000000
3,55000000.000000,13,1994-07-06,677945399.000000,142.000000,Forrest Gump,Comedy,622945399.000000,1,Tom Hanks,Robin Wright,Gary Sinise,Mykelti Williamson,Sally Field,Robert Zemeckis,Eric Roth,39.000000
4,15000000.000000,14,1999-09-15,356296601.000000,122.000000,American Beauty,Drama,341296601.000000,1,Kevin Spacey,Annette Bening,Thora Birch,Wes Bentley,Mena Suvari,Sam Mendes,Alan Ball,54.000000


### Next step... get cast & crew lifetime revenue, profits, # of films, # of profitable films, see notes for other feature ideas

In [226]:
#do same for profits, count of films, count of profitable films, etc.
def get_cast_stats(df, cast_cols):
    '''
    Iterates over a column of cast names (str)
    and generates a new column with the total
    revenue, profit, number of movies, and number of
    profitable movies each person has been in
    up until the date of release in each row.
    --------
    INPUT
    df: dataframe with cast columns
    col_name: list of column names (str) with the cast
    '''
    cols_to_concat = []
    for col_name in cast_cols:
        for i in range(len(df)):
            #getting a dataframe with actor's movies to date only
            name = df.at[i, col_name]
            release_date = df.at[i, 'release_date']
            df_to_sum = df[(df['release_date'] < release_date) & \
                           ((df['cast0'] == name) | (df['cast1'] == name) | \
                            (df['cast2'] == name) | (df['cast3'] == name) | \
                            (df['cast4'] == name))]
            #creating new cols for each actor's lifetime rev, prof, etc. to date
            df.at[i, str(col_name)+'_rev'] = df_to_sum['revenue'].sum()
            df.at[i, str(col_name)+'_prof'] = df_to_sum['profit'].sum()
            df.at[i, str(col_name)+'_count'] = df_to_sum['made_money'].count()
            df.at[i, str(col_name)+'_prof_count'] = df_to_sum['made_money'].sum()

    #summing those cols and dropping them so we have cols representing whole top 5 cast
    rev_cols = ['cast0_rev', 'cast1_rev', 'cast2_rev', 'cast3_rev', 'cast4_rev']
    prof_cols = ['cast0_prof', 'cast1_prof', 'cast2_prof', 'cast3_prof', 'cast4_prof']
    prof_count_cols = ['cast0_prof_count', 'cast1_prof_count', 'cast2_prof_count', 'cast3_prof_count', 'cast4_prof_count']
    tot_count_cols = ['cast0_count', 'cast1_count', 'cast2_count', 'cast3_count', 'cast4_prof']

    rev_to_sum =
    prof_to_sum =
    prof_count_to_sum =
    tot_count_to_sum = 

In [227]:
cast_cols = ['cast0', 'cast1', 'cast2', 'cast3', 'cast4']

#this may take a while to run:
get_cast_stats(df, cast_cols)

In [238]:
df.tail()

,budget,id,release_date,revenue,runtime,title,genre0,profit,made_money,cast0,cast1,cast2,cast3,cast4,director,writer0,releases,cast0_rev,cast0_prof,cast0_count,cast0_prof_count,cast1_rev,cast1_prof,cast1_count,cast1_prof_count,cast2_rev,cast2_prof,cast2_count,cast2_prof_count,cast3_rev,cast3_prof,cast3_count,cast3_prof_count,cast4_rev,cast4_prof,cast4_count,cast4_prof_count
1530,150000000.000000,9982,2005-11-04,314432665.000000,81.000000,Chicken Little,Animation,164432665.000000,1,Zach Braff,Joan Cusack,Steve Zahn,Garry Marshall,Amy Sedaris,Mark Dindal,Steve Bencich,21.000000,26781723.000000,24281723.000000,1.000000,1.000000,1090999573.000000,817999573.000000,5.000000,4.000000,233237516.000000,16737516.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1531,85000000.000000,9986,2006-12-07,144000000.000000,97.000000,Charlotte's Web,Comedy,59000000.000000,1,Dakota Fanning,Julia Roberts,Steve Buscemi,John Cleese,Oprah Winfrey,Gary Winick,Karey Kirkpatrick,6.000000,728949962.000000,496949962.000000,3.000000,2.000000,3177670508.000000,2531670508.000000,14.000000,13.000000,1209231763.000000,674731763.000000,10.000000,8.000000,116567775.000000,29667775.000000,4.000000,3.000000,146292009.000000,131292009.000000,1.000000,1.000000
1532,17500000.000000,9988,2006-08-25,19179969.000000,110.000000,Beerfest,Comedy,1679969.000000,1,Erik Stolhanske,Jay Chandrasekhar,Steve Lemme,Paul Soter,M.C. Gainey,Jay Chandrasekhar,Jay Chandrasekhar,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50159144.000000,14159144.000000,1.000000,1.000000
1533,30000000.000000,9989,2001-01-12,18195610.000000,108.000000,Antitrust,Action,-11804390.000000,0,Ryan Phillippe,Rachael Leigh Cook,Tim Robbins,Claire Forlani,Richard Roundtree,Peter Howitt,Howard Franklin,4.000000,241340984.000000,172340984.000000,5.000000,4.000000,0.000000,0.000000,0.000000,0.000000,166069228.000000,19069228.000000,4.000000,3.000000,174824672.000000,10824672.000000,3.000000,1.000000,25266047.000000,1146047.000000,4.000000,2.000000
1534,86000000.000000,9992,2006-12-13,107944236.000000,94.000000,Arthur and the Invisibles,Adventure,21944236.000000,1,Freddie Highmore,Mia Farrow,Ron Crawford,Penny Balfour,Jason Bateman,Luc Besson,Luc Besson,12.000000,591735319.000000,416735319.000000,2.000000,2.000000,88039551.000000,70519551.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,204999686.000000,152999686.000000,1.000000,1.000000


In [248]:
rev_cols = ['cast0_rev', 'cast1_rev', 'cast2_rev', 'cast3_rev', 'cast4_rev']
prof_cols = ['cast0_prof', 'cast1_prof', 'cast2_prof', 'cast3_prof', 'cast4_prof']
prof_count_cols = ['cast0_prof_count', 'cast1_prof_count', 'cast2_prof_count', 'cast3_prof_count', 'cast4_prof_count']
tot_count_cols = ['cast0_count', 'cast1_count', 'cast2_count', 'cast3_count', 'cast4_prof']


df_2_sum = df[prof_count_cols].copy()


In [249]:
df_2_sum.head()

,cast0_prof_count,cast1_prof_count,cast2_prof_count,cast3_prof_count,cast4_prof_count
0,0.000000,3.000000,1.000000,1.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.000000
2,1.000000,0.000000,0.000000,8.000000,0.000000
3,5.000000,1.000000,0.000000,0.000000,2.000000
4,6.000000,4.000000,1.000000,0.000000,0.000000


#### Sandbox/seeing what some of the data looks like:

In [ ]:
exp_credits_df = pd.read_csv(credits_file)

In [ ]:
# 109, 83, 82, 5, 17, 18, 21 
# Author, Writer

crew_lod = eval(exp_credits_df.iat[21, 2])

In [ ]:
for i in range(len(crew_lod)):
    print(crew_lod[i]['job'])
    print(crew_lod[i]['name'])
    print("")

In [40]:
credits_df.iat[21, 9]

'Michael Herr'

In [117]:
exp_info_df = pd.read_csv(info_file)

In [219]:
col_name = 'cast0'

exp_info_df.at[0, str(col_name)+'_xxx'] = 'testing1234'
### FOR FOR LOOP^^^^^

In [186]:
exp_info_df.iat[0,25]

'testing123'

In [220]:
exp_info_df.head()

,False,/z2QUexmccqrvw1kDMw3R8TxAh5E.jpg,Unnamed: 2,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}]",Unnamed: 5,2,tt0094675,fi,Ariel,"Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned...",4.949,/w0NzAc4Lv6euPtPAmsdEf0ZCF8C.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Villealfa Filmproductions', 'origin_country': 'FI'}]","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0.1,73,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Unnamed: 20,Ariel.1,False.1,7.0,71,xxx,cast0xxx,cast0_xxx
0,False,/6YjUX87VtIEuDzanBE6obVxE9V3.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,3,tt0092149,fi,testing123,"An episode in the life of Nikander, a garbage ...",6.096,/7ad4iku8cYBuB08g9yAU7tHJik5.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1986-10-17,0,72,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Shadows in Paradise,False,7.4,81,testing1234,testing1234,testing1234
1,False,/3EqYpbGCE9S5GddU2K4cYzP5UmI.jpg,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,11.189,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-09,4300000,98,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.3,1121,NaN,NaN,NaN
2,False,/eXKgVIjLCFNpQkjVg1VpA8yM2GA.jpg,NaN,21,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,6,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",9.955,/lNXmgUrP6h1nD53gkFh4WDzT6RZ.jpg,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1993-10-15,12,110,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.5,113,NaN,NaN,NaN
3,False,/udvB86uyzJ6P9vmB83WfrCbnmnI.jpg,NaN,42000,"[{'id': 99, 'name': 'Documentary'}]",http://www.lifeinloops.com,8,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.774,/8YyIjOAxwzD3fZMdmJrfiApod4l.jpg,"[{'id': 19123, 'logo_path': None, 'name': 'inL...","[{'iso_3166_1': 'AT', 'name': 'Austria'}]",2006-01-01,0,80,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.3,6,NaN,NaN,NaN
4,False,NaN,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,9,tt0425473,de,Sonntag im August,NaN,1.332,NaN,[],"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",2004-09-02,0,15,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Sunday in August,False,7.4,4,NaN,NaN,NaN
